# 03_整理資料_過濾詞性

In [1]:
import pandas as pd

df_FromCSVJob104TokenizeData = pd.read_csv('./Data/02_Job104Tokenize.csv', sep='|')
df_FromCSVJob104TokenizeData.head(1)

,Title,Company,County,Salary,Category,JobDescription,Experience,Education,Major,Language,Skills,Others,Url,WS_JobDescription,POS_JobDescription,WS_Others,POS_Others
0,"['東南亞金融科技發展', '商業分析師', '數數發中心,', 'DDT']","['國泰金控', '國泰金融控股股份有限公司']",台北市,"['30000', '90000']","['金融研究員 ', '產品企劃開發人員 ', '其他專案管理師 ']",(1) 規劃與設計以「數據驅動」為核心之商業解決方案，並整合跨單位業務合作以實務落地。\n(...,['2年以上'],"['大學', '碩士']",['不拘'],"['英文', '--', '聽', '/精通', '說', '/精通']",['不拘'],(1) 具備保險或銀行業專業知識者\n(2) 具 SQL 資料庫程式撰寫能力\n(3) 具 ...,https://www.104.com.tw/job/7aflm?jobsource=jol...,"['(1) ', '規劃', '與', '設計', '以', '「', '數據', '驅動'...","['Neu', 'VC', 'Caa', 'Na', 'P', 'PARENTHESISCA...","['(', '1', ') ', '具備', '保險', '或', '銀行業', '專業',...","['PARENTHESISCATEGORY', 'Neu', 'PARENTHESISCAT..."


In [13]:
print(eval(df_FromCSVJob104TokenizeData.iloc[0].WS_JobDescription)[:13])

['(1) ', '規劃', '與', '設計', '以', '「', '數據', '驅動', '」', '為', '核心', '之', '商業']


In [21]:
import re

def FilterWordWithPOS(WS, POS):
    pattern = 'a-zA-Z|\\n|-|\)|\(|【|】|《|》|〈|〉|「|」|『|』|●|◎|■|（|）|–|｜|：|；|。|、|：|，|\d|\.|•|%| '
    list_AllowPos = ['Na', 'VC', 'VE', 'VHC', 'Ncd', 'VA', 'FW', 'Nv', 'VH', 'VE']
    list_JobDescriptionPair = zip(eval(WS), eval(POS))
    list_RowFilteredWords = []
    for word, pos in list_JobDescriptionPair:
        
        word = re.sub(pattern=',', repl=' ', string=word)
        word_en = re.findall(pattern='[a-zA-Z,]{0,100}', string=word)

        word_en = [e for e in word_en if e]
        if word_en:
            list_RowFilteredWords.extend(word_en)
        
        word = re.sub(
            pattern=pattern, 
            repl='', 
            string=word)
    
        if len(word) >= 2 and pos in list_AllowPos:
            list_RowFilteredWords.append(word)
    return list_RowFilteredWords

In [22]:
import re

# list_AllowPos = ['Na', 'VC', 'VE', 'VHC', 'Ncd', 'VA', 'FW', 'Nv', 'VH', 'VE']

list_JobDescriptionPair = zip(df_FromCSVJob104TokenizeData.WS_JobDescription, df_FromCSVJob104TokenizeData.POS_JobDescription)


list_JobDescriptionFilteredWords = []

for row_word, row_pos in list_JobDescriptionPair:
    list_RowFilteredWords = FilterWordWithPOS(row_word, row_pos) 
    list_JobDescriptionFilteredWords.append(list_RowFilteredWords)

In [23]:
list_OthersPair = zip(df_FromCSVJob104TokenizeData.WS_Others, df_FromCSVJob104TokenizeData.POS_Others)
list_OthersFilteredWords = []

for row_word, row_pos in list_OthersPair:
    list_RowFilteredWords = FilterWordWithPOS(row_word, row_pos)
    list_OthersFilteredWords.append(list_RowFilteredWords)

In [24]:
df_FromCSVJob104TokenizeData['Filtered_JobDescription'] = list_JobDescriptionFilteredWords
df_FromCSVJob104TokenizeData['Filtered_Others'] = list_OthersFilteredWords

In [25]:
df_FromCSVJob104TokenizeData.to_csv('./Data/03_Job104Filtered.csv', sep='|', index=False)